In [26]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [27]:
RANDOM_SEED = 17
BATCH_SIZE = 32
np.random.seed(29)

In [28]:
class_mapping = open("inverse_class_mapping",'rb')
inverse_class_label = pickle.load(class_mapping)
class_mapping.close()

In [35]:
class_mapping = open("class_mapping",'rb')
class_label = pickle.load(class_mapping)
class_mapping.close()

In [29]:
def final_fun_1(X):
    tf.keras.backend.clear_session()
    xception = tf.keras.applications.Xception(include_top=False,weights='imagenet',input_shape=(380,380,3))
    xception.trainable = False
    batch_norm = BatchNormalization()
    global_average_pool = GlobalAveragePooling2D()
    dense1 = Dense(256, activation='relu')
    dropout_layer = Dropout(0.5)
    dense2 = Dense(128, activation='relu')
    output =  Dense(120, activation='softmax')
    model = tf.keras.models.Sequential([
        xception,
        batch_norm,
        global_average_pool,
        dense1,
        dropout_layer,
        dense2,
        output
    ])
    model.load_weights("xception.h5")
    df = pd.DataFrame()
    df["id"] = X
    df["id"] = "train/" + df["id"] + ".jpg"
    result = []
    for index, row in df.iterrows():
        img = load_img(row["id"], target_size=(380, 380))
        img = img_to_array(img)
        img = np.true_divide(img, 255)
        img = np.expand_dims(img, axis=0)
        pred = model.predict(img).argmax(axis=-1)
        result.append(inverse_class_label[pred[0]])
    return result

In [30]:
result = final_fun_1(["25526168121fe509d4b323dcb6e23a7b", "841463629c4833816e216cbb041c2778"])

In [31]:
result

['blenheim_spaniel', 'bernese_mountain_dog']

In [40]:
def final_fun_2(X, y):
    tf.keras.backend.clear_session()
    xception = tf.keras.applications.Xception(include_top=False,weights='imagenet',input_shape=(380,380,3))
    xception.trainable = False
    batch_norm = BatchNormalization()
    global_average_pool = GlobalAveragePooling2D()
    dense1 = Dense(256, activation='relu')
    dropout_layer = Dropout(0.5)
    dense2 = Dense(128, activation='relu')
    output =  Dense(120, activation='softmax')
    model = tf.keras.models.Sequential([
        xception,
        batch_norm,
        global_average_pool,
        dense1,
        dropout_layer,
        dense2,
        output
    ])
    model.load_weights("xception.h5")
    df = pd.DataFrame()
    df["id"] = X
    df["id"] = "train/" + df["id"] + ".jpg"
    y_final = [class_label[i] for i in y]
    result = []
    for index, row in df.iterrows():
        img = load_img(row["id"], target_size=(380, 380))
        img = img_to_array(img)
        img = np.true_divide(img, 255)
        img = np.expand_dims(img, axis=0)
        pred = model.predict(img).argmax(axis=-1)
        result.append(pred[0])
    m = tf.keras.metrics.Accuracy()
    m.update_state(result, y_final)
    return m.result().numpy()

In [43]:
result = final_fun_2(["25526168121fe509d4b323dcb6e23a7b", "841463629c4833816e216cbb041c2778"],['blenheim_spaniel', 'saint_bernard'])

In [44]:
result

0.5